In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score

## Read train dataset

In [ ]:
train_data = pd.read_csv("../input/spaceship-titanic/train.csv")

In [ ]:
train_data.columns

## Drop useless data (Passenger ID and Name)

In [ ]:
train_data = train_data.drop(['PassengerId', 'Name'], axis=1)

## Checking if any NAN data
**if there has any NAN data, drop that row**

In [ ]:
np.sum(train_data.isna())

In [ ]:
train_data = train_data.dropna()

In [ ]:
np.sum(train_data.isna())

## Checking some variables

In [ ]:
train_data.HomePlanet.value_counts()

In [ ]:
train_data.CryoSleep.value_counts()

In [ ]:
train_data.VIP.value_counts()

In [ ]:
train_data.Transported.value_counts()

## visualization

In [ ]:
HomePlanet_transported = pd.crosstab(train_data.HomePlanet, train_data.Transported)
HomePlanet_transported.plot(figsize= (20, 10), kind = 'bar')
plt.title('Relationship between Home Planet and Transported')
plt.xlabel('Home Planet')
plt.ylabel('Transported')

In [ ]:
CryoSleep_transported = pd.crosstab(train_data.CryoSleep, train_data.Transported)
CryoSleep_transported.plot(figsize= (20, 10), kind = 'bar')
plt.title('Relationship between CryoSleep and Transported')
plt.xlabel('CryoSleep')
plt.ylabel('Transported')

In [ ]:
VIP_transported = pd.crosstab(train_data.VIP, train_data.Transported)
VIP_transported.plot(figsize= (20, 10), kind = 'bar')
plt.title('Relationship between VIP and Transported')
plt.xlabel('VIP')
plt.ylabel('Transported')


In [ ]:
age_transported = pd.crosstab(train_data.Age, train_data.Transported)
age_transported.plot(figsize= (20, 10), kind = 'bar')
plt.title('Relationship between Age and Transported')
plt.xlabel('Age')
plt.ylabel('Transported')


## Convert category variables to numeric variables

In [ ]:
for col_name in train_data.columns:
    if(train_data[col_name].dtype == 'object'):
        train_data[col_name]= train_data[col_name].astype('category')
        train_data[col_name] = train_data[col_name].cat.codes

display(train_data)

## Split train dataset

In [ ]:
X = train_data.drop(["Transported"], axis=1)
y = train_data.Transported
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Build logistic regression model and display confusion matirx

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=10000).fit(X_train, y_train)
ypred = clf.predict(X_test)
cm = confusion_matrix(y_test, ypred, labels=clf.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_).plot()


In [ ]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, clf.predict(X_test))))

**Show statistical report**

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, ypred))

## ROC

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, clf.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()


## Finding most related variables

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(train_data.Transported, train_data.drop(["Transported"], axis=1))
result=logit_model.fit()
print(result.summary())

In [ ]:
test_data = pd.read_csv("../input/spaceship-titanic/test.csv")
test_data = test_data.drop(['PassengerId', 'Name'], axis=1)
test_data=test_data.apply(lambda x: x.fillna(x.value_counts().index[0]))
test_data.info()

In [ ]:
for col_name in test_data.columns:
    if(test_data[col_name].dtype == 'object'):
        test_data[col_name]= test_data[col_name].astype('category')
        test_data[col_name] = test_data[col_name].cat.codes

display(test_data)

In [ ]:
test_pred_LR = clf.predict(test_data)

In [ ]:
sample_submission= pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
sample_submission.Transported=test_pred_LR
LR_results=sample_submission
print(LR_results)

In [ ]:
LR_results.to_csv("sample_submission.csv",index=False)
print("Completed")